In [3]:
from numpy import hstack
from numpy import zeros
from numpy import ones
import numpy as np
from numpy.random import rand
from numpy.random import randn
from keras.datasets import mnist, fashion_mnist
from keras.optimizers import Adam
from keras.losses import mse, binary_crossentropy
from keras.models import Sequential
from keras.models import Model
from keras.layers import *
from numpy.random import randint
from matplotlib import pyplot as plt
import os

def define_discriminator():
    inputs = Input(shape=( ... )) # discriminator input的維度必須等於generator output及圖片的大小
    ''''
    請建立discriminator網路
    ''''
    outputs = ...  # discriminator output 的維度等於 1
    
    model = Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    inputs = Input(shape=( ... )) # generator input 的維度必須等於latent_dim
    ''''
    請建立generator網路
    ''''
    outputs = ... # generator output 的維度必須等於圖片的大小(28*28)
    
    model = Model(inputs, outputs)
    return model

def define_gan(generator, discriminator):
    #將discriminator的參數設成不可訓練
    discriminator.trainable = False
    
    #連接generator, discriminator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# 生成 n 個真實樣本和標籤
def load_real_samples(n):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    'max-min narmalize:將數值標準化成0-1'

    'reshape data size:將圖片轉成一維(28, 28) -> (784)'
    
    return x_train, y_train, x_test, y_test

def generate_real_samples(dataset, n):
    images = dataset
    ix = randint(0, images.shape[0], n)
    x = images[ix]
    y = ones((n, 1))
    return x, y

# 生成latent_space中的點作為生成器的輸入
def generate_latent_points(latent_dim, n):
    # 在latent_space中生成點
    x_input = randn(latent_dim * n)
    
    x_input = x_input.reshape(n, latent_dim)
    return x_input

# 用生成器生成 n 個假樣本和類標籤
def generate_fake_samples(generator, latent_dim, n):
    # 在letent_space中生成點
    x_input = generate_latent_points(latent_dim, n)
    # 預測輸出值
    X = generator.predict(x_input)
    # 創建假標籤
    y = zeros((n, 1))
    return X, y

# 評估判別器並且繪製真假點
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # 準備真實樣本
    x_real, y_real = generate_real_samples(n)
    # 在真實樣本上評估判別器
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # 準備假樣本
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # 在假樣本上評估判別器
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # 總結判別器性能
    print(epoch, acc_real, acc_fake)
    # 繪製真假數據的散點圖
    pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    pyplot.show()
    
def train(g_model, d_model, gan_model, latent_dim, n_epochs=..., n_batch=..., n_eval=...):
    # 用一半的 batch 來訓練判別器
    half_batch = int(n_batch / 2)
    
    x_train, _, x_test, y_test = load_real_samples()
    for i in range(n_epochs):
        #生成真假樣本
        x_real, y_real = generate_real_samples(...)
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, ...)
        
        for j in range(...):
            #先訓練discriminator
            d_model.train_on_batch( ... , ... )
            d_model.train_on_batch( ... , ... )
        
        for j in range(...):
            #生成隨機的值給generator input
            x_gan = generate_latent_points(latent_dim, ...)
            y_gan = ...

            #訓練generator騙過discriminator
            gan_model.train_on_batch( ... , ... )
        
        if i % ... == 0:
            sample_images(g_model, latent_dim)
            
latent_dim = 5
# 建立判別器
discriminator = define_discriminator()
# 建立生成器
generator = define_generator(latent_dim)
# 建立生成對抗網絡
gan_model = define_gan(generator, discriminator)
# 訓練
train(generator, discriminator, gan_model, latent_dim)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

C:\Users\JohnnyHu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.771445, acc.: 43.75%] [G loss: 0.606311]


C:\Users\JohnnyHu\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.382319, acc.: 87.50%] [G loss: 0.681662]
2 [D loss: 0.339543, acc.: 85.94%] [G loss: 0.710248]
3 [D loss: 0.319309, acc.: 93.75%] [G loss: 0.800306]
4 [D loss: 0.306121, acc.: 92.19%] [G loss: 0.864751]
5 [D loss: 0.297015, acc.: 90.62%] [G loss: 0.970128]
6 [D loss: 0.280640, acc.: 95.31%] [G loss: 1.112023]
7 [D loss: 0.213455, acc.: 100.00%] [G loss: 1.235536]
8 [D loss: 0.187587, acc.: 100.00%] [G loss: 1.450482]
9 [D loss: 0.146232, acc.: 100.00%] [G loss: 1.626104]
10 [D loss: 0.131943, acc.: 100.00%] [G loss: 1.718402]
11 [D loss: 0.112751, acc.: 100.00%] [G loss: 1.894839]
12 [D loss: 0.105577, acc.: 100.00%] [G loss: 2.107992]
13 [D loss: 0.090238, acc.: 100.00%] [G loss: 2.280482]
14 [D loss: 0.070830, acc.: 100.00%] [G loss: 2.388279]
15 [D loss: 0.066313, acc.: 100.00%] [G loss: 2.338377]
16 [D loss: 0.061200, acc.: 100.00%] [G loss: 2.656862]
17 [D loss: 0.050395, acc.: 100.00%] [G loss: 2.586343]
18 [D loss: 0.052206, acc.: 100.00%] [G loss: 2.654156]
19 [D l

...

29186 [D loss: 0.651723, acc.: 62.50%] [G loss: 1.012451]
29187 [D loss: 0.677409, acc.: 56.25%] [G loss: 0.964542]
29188 [D loss: 0.572469, acc.: 73.44%] [G loss: 1.209104]
29189 [D loss: 0.673280, acc.: 62.50%] [G loss: 0.953735]
29190 [D loss: 0.620540, acc.: 67.19%] [G loss: 0.832854]
29191 [D loss: 0.699472, acc.: 62.50%] [G loss: 0.830321]
29192 [D loss: 0.599585, acc.: 65.62%] [G loss: 0.953523]
29193 [D loss: 0.598537, acc.: 70.31%] [G loss: 0.870258]
29194 [D loss: 0.600519, acc.: 62.50%] [G loss: 1.040819]
29195 [D loss: 0.643389, acc.: 59.38%] [G loss: 0.961256]
29196 [D loss: 0.636400, acc.: 64.06%] [G loss: 1.022078]
29197 [D loss: 0.610296, acc.: 68.75%] [G loss: 0.986562]
29198 [D loss: 0.607680, acc.: 71.88%] [G loss: 1.029822]
29199 [D loss: 0.653063, acc.: 62.50%] [G loss: 1.014831]
29200 [D loss: 0.685819, acc.: 56.25%] [G loss: 1.022602]
29201 [D loss: 0.655898, acc.: 60.94%] [G loss: 0.957447]
29202 [D loss: 0.658932, acc.: 65.62%] [G loss: 0.997356]
29203 [D loss:

29330 [D loss: 0.626612, acc.: 62.50%] [G loss: 1.095408]
29331 [D loss: 0.643116, acc.: 62.50%] [G loss: 1.153676]
29332 [D loss: 0.684331, acc.: 57.81%] [G loss: 0.997221]
29333 [D loss: 0.718689, acc.: 59.38%] [G loss: 0.948668]
29334 [D loss: 0.666688, acc.: 56.25%] [G loss: 0.943971]
29335 [D loss: 0.654546, acc.: 64.06%] [G loss: 1.029621]
29336 [D loss: 0.682954, acc.: 54.69%] [G loss: 0.972619]
29337 [D loss: 0.607733, acc.: 64.06%] [G loss: 0.997891]
29338 [D loss: 0.652367, acc.: 60.94%] [G loss: 1.024933]
29339 [D loss: 0.601923, acc.: 68.75%] [G loss: 1.042763]
29340 [D loss: 0.583960, acc.: 67.19%] [G loss: 1.038359]
29341 [D loss: 0.648779, acc.: 60.94%] [G loss: 0.989988]
29342 [D loss: 0.681438, acc.: 62.50%] [G loss: 0.955777]
29343 [D loss: 0.647638, acc.: 64.06%] [G loss: 1.010618]
29344 [D loss: 0.659291, acc.: 64.06%] [G loss: 0.962790]
29345 [D loss: 0.657749, acc.: 62.50%] [G loss: 1.070099]
29346 [D loss: 0.651694, acc.: 62.50%] [G loss: 1.042061]
29347 [D loss:

29472 [D loss: 0.602816, acc.: 65.62%] [G loss: 1.039673]
29473 [D loss: 0.691419, acc.: 46.88%] [G loss: 1.049967]
29474 [D loss: 0.691076, acc.: 62.50%] [G loss: 1.048171]
29475 [D loss: 0.609904, acc.: 60.94%] [G loss: 0.961260]
29476 [D loss: 0.583684, acc.: 68.75%] [G loss: 1.039816]
29477 [D loss: 0.654286, acc.: 62.50%] [G loss: 1.063641]
29478 [D loss: 0.696458, acc.: 57.81%] [G loss: 0.926829]
29479 [D loss: 0.639718, acc.: 62.50%] [G loss: 1.030326]
29480 [D loss: 0.624818, acc.: 67.19%] [G loss: 1.021715]
29481 [D loss: 0.566326, acc.: 70.31%] [G loss: 1.007791]
29482 [D loss: 0.648680, acc.: 65.62%] [G loss: 1.000348]
29483 [D loss: 0.657044, acc.: 60.94%] [G loss: 0.946649]
29484 [D loss: 0.546234, acc.: 75.00%] [G loss: 1.033534]
29485 [D loss: 0.583297, acc.: 71.88%] [G loss: 0.992581]
29486 [D loss: 0.663709, acc.: 56.25%] [G loss: 0.978356]
29487 [D loss: 0.690584, acc.: 57.81%] [G loss: 0.899064]
29488 [D loss: 0.651352, acc.: 65.62%] [G loss: 0.950919]
29489 [D loss:

29765 [D loss: 0.617875, acc.: 65.62%] [G loss: 1.044409]
29766 [D loss: 0.719058, acc.: 50.00%] [G loss: 1.104535]
29767 [D loss: 0.576438, acc.: 78.12%] [G loss: 1.070359]
29768 [D loss: 0.655270, acc.: 54.69%] [G loss: 1.088497]
29769 [D loss: 0.719777, acc.: 54.69%] [G loss: 1.030650]
29770 [D loss: 0.686895, acc.: 51.56%] [G loss: 1.090292]
29771 [D loss: 0.615394, acc.: 62.50%] [G loss: 1.046120]
29772 [D loss: 0.676907, acc.: 60.94%] [G loss: 0.941862]
29773 [D loss: 0.662807, acc.: 62.50%] [G loss: 1.064539]
29774 [D loss: 0.653678, acc.: 68.75%] [G loss: 1.041679]
29775 [D loss: 0.706624, acc.: 57.81%] [G loss: 1.037381]
29776 [D loss: 0.717168, acc.: 56.25%] [G loss: 0.856405]
29777 [D loss: 0.645380, acc.: 56.25%] [G loss: 1.135906]
29778 [D loss: 0.641068, acc.: 60.94%] [G loss: 1.201561]
29779 [D loss: 0.645299, acc.: 57.81%] [G loss: 1.125521]
29780 [D loss: 0.670914, acc.: 60.94%] [G loss: 0.996898]
29781 [D loss: 0.703765, acc.: 54.69%] [G loss: 1.023951]
29782 [D loss:

29917 [D loss: 0.612634, acc.: 67.19%] [G loss: 1.115988]
29918 [D loss: 0.623522, acc.: 65.62%] [G loss: 0.921547]
29919 [D loss: 0.638023, acc.: 67.19%] [G loss: 0.927031]
29920 [D loss: 0.651639, acc.: 56.25%] [G loss: 1.033117]
29921 [D loss: 0.606913, acc.: 65.62%] [G loss: 0.934664]
29922 [D loss: 0.618307, acc.: 67.19%] [G loss: 1.018329]
29923 [D loss: 0.596088, acc.: 64.06%] [G loss: 0.938233]
29924 [D loss: 0.677189, acc.: 56.25%] [G loss: 0.872901]
29925 [D loss: 0.642314, acc.: 70.31%] [G loss: 1.039709]
29926 [D loss: 0.507771, acc.: 76.56%] [G loss: 1.061158]
29927 [D loss: 0.614146, acc.: 57.81%] [G loss: 0.973158]
29928 [D loss: 0.664218, acc.: 56.25%] [G loss: 0.891798]
29929 [D loss: 0.631192, acc.: 64.06%] [G loss: 0.818663]
29930 [D loss: 0.603707, acc.: 71.88%] [G loss: 0.975511]
29931 [D loss: 0.669204, acc.: 62.50%] [G loss: 1.042993]
29932 [D loss: 0.640828, acc.: 64.06%] [G loss: 0.982253]
29933 [D loss: 0.628564, acc.: 64.06%] [G loss: 1.088226]
29934 [D loss: